In [2]:
%load_ext autoreload

import numpy as np
import pickle
import cv2
import glob

car_images = glob.glob('../vehicles/**/*.png')
noncar_images = glob.glob('../non-vehicles/**/*.png')
print('Number of car images', len(car_images))
print('Number of non-car images', len(noncar_images))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Number of car images 8792
Number of non-car images 8968


In [ ]:
%autoreload

from tqdm import tqdm
from image_features import ImageFeatures
from vehicle_classifier import VehicleClassifier

best_score = 0
best_params = {}
best_classifier = None

with tqdm(total=28800) as pbar:
    for cspace in ['RGB', 'HSV', 'LUV', 'HLS', 'YUV', 'YCrCb']:
        for bin_size in [(8,8), (16,16), (32,32), (48,48)]:
            for hist_bins in [8, 16, 32, 48]:
                for hist_bin_range in [8, 16, 32, 48]:
                    for hog_orient in [9, 10, 11, 12, 13]:
                        for hog_pix_per_cell in [4, 6, 8, 10, 12]:
                            for hog_cell_per_block in [2, 4, 6]:
                                featurizer = ImageFeatures(color_space=cspace, 
                                                 bin_size=bin_size, 
                                                 hist_bins=hist_bins, 
                                                 hist_bin_range=(0,hist_bin_range), 
                                                 hog_orient=hog_orient, 
                                                 hog_pix_per_cell=hog_pix_per_cell, 
                                                 hog_cell_per_block=hog_cell_per_block)
                                car_features = featurizer.featurize(car_images)
                                noncar_features = featurizer.featurize(noncar_images)
                                classifier = VehicleClassifier(car_features, noncar_features)
                                score = classifier.fit()
                                if (score > best_score):
                                    print('Classification accuracy improved to: ', score)
                                    best_score = score
                                    best_classifier = classifier
                                    best_params['cspace'] = cspace
                                    best_params['bin_size'] = bin_size
                                    best_params['hist_bins'] = hist_bins
                                    best_params['hist_bin_range'] = hist_bin_range
                                    best_params['hog_orient'] = hog_orient
                                    best_params['hog_pix_per_cell'] = hog_pix_per_cell
                                    best_params['hog_cell_per_block'] = hog_cell_per_block
                                
                                pbar.update(1)

print('Best accuracy = ', best_accuracy)
print("Best parameters are: Color Space =  %s, bin_size = %s, hist_bins = %d, hist_bin_range = (0, %d), hog_orient = %d, hog_pix_per_cell = %d, hog_pix_per_block = %d" % (best_params['cspace'], best_params['bin_size'], best_params['hist_bins'], best_params['hist_bin_range'], best_params['hog_orient'], best_params['hog_pix_per_cell'], best_params['hog_cell_per_block']))

  0%|          | 0/28800 [00:00<?, ?it/s]

In [24]:
%autoreload 
import tqdm

print('Classifier score:', 10)

Classifier score: 10


In [21]:
print("%d" % 10)

10
